#Máster en Big Data y Data Science: Ciencia e Ingeniería de Datos
### ASIGNATURA: Indexación, búsqueda y análisis en repositorios multimedia
### PARTE: Multimedia (imagen, video)
### Práctica 1: Introducción al diseño de redes neuronales convolucionales con Pytorch mediante Google Colaboratory

---

Autor: Juan C. SanMiguel (juancarlos.sanmiguel@uam.es), Universidad Autónoma de Madrid


# 1. Setup
En este script interativo de python aprendera como utilizar la herramienta Google Colaboratory https://colab.research.google.com 

Colaboratory permite utilizar gratuitamente una máquina virtual con Ubuntu que proporciona un entorno de Jupyter para ejecutar interactivamente instrucciones en una consola de Python. 

Información adicional (política de uso, restricciones,...) disponible en https://research.google.com/colaboratory/faq.html

## Ejecutar instrucciones de sistema

Utilize el símbolo '!' delante de cada instrucción de sistema que desee ejecutar. Como por ejemplo la instruccion 'ls' muestra el contenido del directorio actual

```
$ !ls
```
Por ejemplo, ejecute cualquiera de las siguientes instrucciones para analizar los detalles de la máquina virtual ejecutando este entorno de Jupyter

In [ ]:
# Show linux release
!lsb_release -a

In [ ]:
# Show CPU info
!cat /proc/cpuinfo

In [ ]:
# Show RAM info
!cat /proc/meminfo

In [ ]:
# Show mounted directories 
!df -h

En esta práctica, vamos a verificar la versión de Python que estamos ejecutando (recuerde que el código de esta práctica se ha comprobado sobre Python 2.7)

In [ ]:
#Check Python version
import sys
sys.version

## Verificar existencia de GPU
Una de las principales ventajas de *Google Colaboratory* es la disponibilidad gratuita de recursos de calculo intensivo mediante GPUs (Graphic Processing Unit). En esta sección describiremos como verificar la existencia de esta GPU.

Primeramente, debe activar el recurso GPU. Para ello navege por el menu "Edit-->Notebook Settings" y seleccione la opción "GPU" en la parte "Hardware acceleretor". En la otra parte "Runtime type", seleccione "Python3" pues que todo el código de esta práctica ha sido verificado con Python 3. 

A continuación se muestra un ejemplo visual de las opciones en "Notebook Settings"

![alt text](https://cdn-images-1.medium.com/max/1600/1*WNovJnpGMOys8Rv7YIsZzA.png)

Posteriormente vamos a utilizar la instrucción  *nvidia-smi* para identificar cuantas GPUs existen en nuestro sistema:

In [ ]:
!nvidia-smi

## Instalacion de paquetes necesarios

En esta sección vamos a mostrar el procedimiento para instalar el software necesario para realizar esta práctica

### Jupyter notebook
En el caso de que la máquina virtual de https://colab.research.google.com/ no esté disponible (puede verificar que aparece la palabra "CONNECTED" arriba a la derecha de su pantalla). Se sugiere realizar la práctica en local (e.g. PC o portátil personal). Para ejecutar los scripts interactivos \*.ipynb, necesitará instalar *Jupyter*. 

Ejecute los siguientes comandos en un terminal de linux


```
$ pip install jupyter #version 2.X python (recomendada)
$ pip3 install jupyter #version 3.X python
$ jupyter notebook # lanza jupyter y abre un navegador para ejecutar scripts ipynb
```

El procedimiento completo está descrito en http://jupyter.readthedocs.io/en/latest/install.html

NOTA: los scripts ipynb de esta práctica están preparados para funcionar tanto en local como en colab.research.google.com (asumiendo una conexión directa a internet, sin proxy alguno).


### Pytorch 
En esta práctica utilizaremos ```Pytorch``` para las tareas de *deep learning* y el paquete ``torchvision`` que proporciona funcionalidad extendida. 

A continuación se muestran las instrucciones necesarias para instalar Pytorch con/sin soporte para cálculo intensivo mediante GPUs (descomente aquella opción que desee instalar):

In [ ]:
#install pytorch for python 2.7 without CUDA support
#!pip install torchvision==0.2.0
#!pip install http://download.pytorch.org/whl/cpu/torch-0.3.1-cp27-cp27mu-linux_x86_64.whl 

#install pytorch for python 2.7 with CUDA 8.0 support
!pip install torchvision==0.2.0
!pip install http://download.pytorch.org/whl/cu80/torch-0.3.1-cp27-cp27mu-linux_x86_64.whl 

### Pillow

También utilizaremos la librería ``Pillow`` para lectura/escritura de imágenes. En esta práctica utilizaremos la versión 5.0.0 debido a que 
la versión 4.0.0 tiene problemas de compatibilidad con Colaborative (en concreto para la lectura de imágenes con plugins de lectura *TiffImageFile*)

In [ ]:
!pip install Pillow==5.0.0
!pip show pillow #check pillow version

Para actualizar el entorno y se utilice la versión 5.0 del paquete ``Pillow``, deberá resetear el *runtime* de esta sesión de ``Jupyter``.

Para ello, utilice la opción del menu "Runtime"->"Restart Runtime" ó seleccione Ctrl+M.

## Conectarse a sus carpetas en google drive
Debido a que el uso del entorno *colab.research.google.com* está registringido a un máximo de 12 horas continuadas (tras las cuales, se reseteará su máquina virtual y podrá volver a utilizar otra distinta) se sugiere conectar esta máquina virtual a la unidad de *Google Drive* asociada a su usuario con cuenta @gmail. Así podrá guardar datos y resultados que genere para un posterior uso.

Debido a que el procedimiento descrito a continuación requiere acceder a su cuenta personal @gmail, **se sugiere crear una nueva cuenta @gmail para la realización de esta práctica**.

Tras identificarse con su nueva cuenta @gmail, vuelva a cargar este script refrescando esta página web en el navegador.


A continuación utilizaremos el paquete *drive* de *Google.Colab* que permite montar sus directorios de Drive en la carpeta del sistema */content/gdrive/*

Estas instrucciónes le pediran autorizar el acceso a su cuenta proporcionándole un enlace. Haga click sore el enlace y copie en el código que obtenga en el cuadro de texto.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Podemos visualizar el contenido de nuestra carpeta raíz en Google Drive con la siguiente instrucción

In [ ]:
!ls /content/gdrive/My\ Drive/

A continuación creamos un fichero de ejemplo para verificar el correcto funcionamiento de las instrucciones anteriores



In [ ]:
with open('/content/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/gdrive/My\ Drive/foo.txt

Por último podemos crear un carpeta para guardar los datos de esta práctica

In [ ]:
!mkdir /content/gdrive/My\ Drive/practica1indexacion

## Operaciones comúnes

En esta sección repasaremos operaciones/comandos de utilidad para el resto de partes de la práctica

### Moverse a otros directorios

El directorio de trabajo por defecto es '/content'. Si hemos seguido el paso anterior, lo deseable es que trabajemos en nuestros directorios de *Google Drive*. 

Para movernos entre directorios normalmente utilizariamos la instrucción **cd**. De manera alternativa se sugiere utilizar la libreria *os* para moverse entre directorios.


In [ ]:
#show current directory
!pwd

#move to a specific directory
import os
os.chdir('/content/gdrive/My Drive/practica1indexacion/')
!pwd

### Generación de scripts

Frecuentemente, tras desarrollar una nueva funcionalidad es conveniente utilizar un fichero *.py* para encapsular esta funcionalidad.

En el siguiente código puede ver como se generan el fichero *textproc.py*

In [ ]:
%%writefile textproc.py

def plural(word):
    if word.endswith('y'):
        return word[:-1] + 'ies'
    elif word[-1] in 'sx' or word[-2:] in ['sh', 'ch']:
        return word + 'es'
    elif word.endswith('an'):
        return word[:-2] + 'en'
    else:
        return word + 's'

In [ ]:
#list directory contents
!ls

### Ejecución de scripts

Tras encapsular la funcionalidad deseada en un fichero *.py*, tenemos dos maneras utilizar dicha funcionalidad

In [ ]:
#example 1 for importing library
from textproc import plural

#create object
plural('wish') 

In [ ]:
#example 2 for running an external script

%run -i textproc.py #runs script and keeps variables
plural('wish')

### Reiniciar la máquina virtual
Es posible que realice diversas operaciones sobre el sistema (e.g. instalación de paquetes) que produzca un sistema inestable. 

En ese caso, se recomienda:

1) Haga un "restart runtime" mediante las teclas Ctrl+M

2) En el caso de que no se resuelva su problema, ejecute la siguiente instrucción para forzar un reseteo del sistema. Tras ejecutar esta instrucción la máquina sufrirá un bloqueo durante un pequeño lapso de tiempo (1-2 minutos). Refresque la página y verifique que se conecta a la máquina virtual ("CONNECTED" en la esquina superior derecha)

```
!kill -9 -1
```
   




In [ ]:
!kill -9 -1